In [3]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')
import pandas as pd, numpy as np
from utils.reductors import *
from utils.drplotter import *
from collections import namedtuple
hue_info = namedtuple('hue_info', ['field_name', 'is_categorical'])

# Считаем крабов для примера работы рисовалки
tr = pd.read_csv('crabs_train.csv')

tr.columns = tr.columns.str.lower()
tr.rename(columns={
    'shucked weight': 'shucked_weight',
    'viscera weight': 'viscera_weight',
    'shell weight': 'shell_weight',
}, inplace=True)

drp = DimReductionPlotter()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# никаких боссов (journey mode)

- понимая, что шансов успеть все нет, я решил все время уделить рисовалкам (возможно, потыкаю курильщиков на каникулах)
- но рисовалки выглядит как куда более привлекательнее, т.к. a) задания по типу "эта штука вам пригодится" - самые лучшие на любых курсах с прогой (or smth) и б) кажется, здесь можно большему научиться из-за большей детерминированности процесса/результата

Что реализовано?

## все вместе, без объяснений (you can skip this part)

In [89]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd, numpy as np
from utils.reductors import *
from utils.drplotter import *
from collections import namedtuple
hue_info = namedtuple('hue_info', ['field_name', 'is_categorical'])

from sklearn.datasets import load_wine
from sklearn.preprocessing import StandardScaler
from functools import partial

df = load_wine()
df = pd.DataFrame(
  np.concatenate([df['data'],df['target'].reshape(-1,1)], axis=1),
  columns=df['feature_names']+['target']
)
non_scaled_cols = [i for i in df.columns if i != 'target']
scaled_cols = ['scaled_'+i for i in non_scaled_cols]
df.loc[:, scaled_cols] = StandardScaler().fit_transform(df.drop(columns='target'))


drp = DimReductionPlotter()


mapper_dict = {
    'tsne 2D, init=pca, affinities: perplexity=2, k_neighbors=4': {
        'params': {
           'n_jobs': 1,
           'verbose': False,
           'n_components': 2,
        },
        'func': partial(make_tsne,  
          affinities=openTSNE.affinity.PerplexityBasedNN(df[scaled_cols], perplexity=2, k_neighbors=4, n_jobs=1)
        ),
        'hue': hue_info('target', True),
        'features': scaled_cols,
        'hover_data': non_scaled_cols,
    },
    'tsne 2D, init=pca, no affinities': {
        'params': {
           'n_jobs': 1,
           'verbose': False,
           'n_components': 2,
        },
        'func': make_tsne,
        'hue': hue_info('target', True),
        'features': scaled_cols,
        'hover_data': non_scaled_cols,
    },
    'tsne 2D, init=random, no affinities': {
        'params': {
           'n_jobs': 1,
           'verbose': False,
           'n_components': 2,
        },
        'func': partial(make_tsne, init=openTSNE.initialization.random(df[scaled_cols].values)),
        'hue': hue_info('target', True),
        'features': scaled_cols,
        'hover_data': non_scaled_cols,
    },
    'UMAP 2D, min_dist=0.05': {
      'params': {
        'n_neighbors': 10,
        'min_dist': 0.05,
        'n_jobs': 1,
        'verbose': False,
        'n_components': 2
      },
      'func': make_umap,

      'hue': hue_info('target', True),
      'features': scaled_cols,
      'hover_data': non_scaled_cols,
    }, 
    'UMAP 2D, min_dist=0.5': {
      'params': {
        'n_neighbors': 10,
        'min_dist': 0.5,
        'n_jobs': 1,
        'verbose': False,
        'n_components': 2
      },
      'func': make_umap,

      'hue': hue_info('target', True),
      'features': scaled_cols,
      'hover_data': non_scaled_cols,
    }, 
    'PCA 3D, no menu': {
        'params': {
           'n_components': 3
        },
        'func': make_pca,
        'hue': hue_info('target', True),
        'features': scaled_cols,
        'hover_data': non_scaled_cols+['target'],
    },
    'PCA 3D, with menu': {
        'params': {
           'n_components': 3
        },
        'func': make_pca,
        
        'features': scaled_cols,
        'hover_data': non_scaled_cols+['target'],
        'hue_cols': non_scaled_cols+['target'],
        'cat_trsh': 5,
    },
    'SPECTRAL 3D': {
        'params': {
           'n_components': 3
        },
        'func': make_spectral,

        'features': scaled_cols,
        'hover_data': non_scaled_cols+['target'],
        'hue_cols': non_scaled_cols+['target'],
        'cat_trsh': 5,
    },
}

drp.plot_dim_reduction(
  data = df,
  mapper_dict=mapper_dict,
  default_features=non_scaled_cols,
  default_hue_info=None,
  row_height=450,
  row_width=1200,
  reuse_results=False
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 8/8 [00:04<00:00,  1.90it/s]


Loading BokehJS ...

    'data': [{'customdata': array([[14.23, 1.71, 2.43, ..., 3.92,…

## 1. решетка графиков и spectral_embedding
комментарии не нужны

In [6]:
mapper_dict = {
    'tsne1': {
      'params': {
        'n_jobs': 1,
        'verbose': False,
        'n_components': 2
      },
      'func': make_tsne,
    },
    'tsne2': {
      'params': {
        'n_jobs': 1,
        'verbose': False,
        'n_components': 2
      },
      'func': make_tsne,
    },
    'tsne3': {
      'params': {
        'n_jobs': 1,
        'verbose': False,
        'n_components': 3
      },
      'func': make_tsne,
    },
    'UMAP1': {
      'params': {
        'n_neighbors': 11,
        'min_dist': 0.1,
        'n_jobs': 1,
        'verbose': False,
        'n_components': 2
      },
      'func': make_umap,
    },
    'UMAP2': {
      'params': {
        'n_neighbors': 11,
        'min_dist': 0.1,
        'n_jobs': 1,
        'verbose': False,
        'n_components': 2
      },
      'func': make_umap,
    },
    'UMAP3': {
      'params': {
        'n_neighbors': 11,
        'min_dist': 0.1,
        'n_jobs': 1,
        'verbose': False,
        'n_components': 3
      },
      'func': make_umap,
    },
    'PCA 3D': {
        'params': {
           'n_components': 3
        },
        'func': make_pca,
        'features': ['weight', 'shucked_weight', 'shell_weight'],
    },
    'SPECTRAL 3D': {
        'params': {
           'n_components': 3
        },
        'func': make_spectral,
        'features': ['weight', 'shucked_weight', 'shell_weight'],
    },
}

drp.plot_dim_reduction(
  data = tr.sample(1_000),
  mapper_dict=mapper_dict,
  default_features=tr.drop(columns=['sex', 'age']).columns,
  default_hue_info=hue_info(field_name='sex', is_categorical=True), 
  row_height=450,
  row_width=1200,
  reuse_results=False
)

  0%|          | 0/8 [00:00<?, ?it/s]

100%|██████████| 8/8 [00:36<00:00,  4.57s/it]


    'data': [{'hovertemplate': 'sex=M<br>x=%{x}<br>y=%{y}<br>z=%{…

## 2. hover_data
- не понял как убирать x,y,z, из plotly как ни пытался(((

In [21]:
mapper_dict = {
  'tsne': {
    'params': {
      'n_jobs': 1,
      'verbose': False,
      'n_components': 2
    },
    'func': make_tsne,
    'hover_data': ['sex', 'age', 'weight'],
  },
  'PCA 3D': {
    'params': {
        'n_components': 3
    },
    'func': make_pca,
    'features': ['weight', 'shucked_weight', 'shell_weight'],
    'hover_data': ['sex', 'age', 'weight'],
  },
}

drp.plot_dim_reduction(
  data = tr.sample(1_000),
  mapper_dict=mapper_dict,
  default_features=tr.drop(columns=['sex', 'age']).columns,
  default_hue_info=hue_info(field_name='sex', is_categorical=True), 
  row_height=500,
  row_width=1000,
  reuse_results=False
)

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:07<00:00,  3.58s/it]


    'data': [{'customdata': array([['I', 7, 13.79203175],
       …

## 3. динамическая прозрачность (нет конечно)

- динамическая прозрачность для plotly должна стоить как минимум 6 баллов (рандомное большое число), так как во-первых plotly не поддерживает прозрачность не в виде одного-единственного числа, так во-вторых когда я пол дня придумывал способ расставить всем точкам цвета (цвета можно массивом передавать, а прозрачность - нет) в зависимости от hue_name и opacity_col, но получился какой-то бред: одинаковые цвета с разными opacity раскрашиваются в разные цвета (оно перемешивается непонятным образом из-за коллизии с color=hue_name, то есть нормально прозрачность отображается когда не указан color=hue_name и я просто решил убить эту фичу). с bokeh очень схожая история: нигде в интернете/gpt не нашел каких-либо использований динамической прозрачности... видимо, это должен быть javascript script и я не знаю как с этим работать

In [ ]:
# для справки такой костыль можно использовать для plotly opacity, но это все слишком костыльно

self.hue_rgb = np.array(list(map(px.colors.hex_to_rgb, px.colors.qualitative.D3)))
color = np.zeros((plot_data.shape[0], 4))
if opacity_col is not None:
  if hue_name is not None:
    color[:, :-1] = self.hue_rgb[pd.factorize(hue)[0]]
  else:
    color[:, :-1] = self.hue_rgb[:1] * plot_data.shape[0]
  color[:, -1]  = (opacity_col - min(opacity_col)) / (max(opacity_col) - min(opacity_col)) * (1 - min_opacity) + min_opacity
  color = list(map(lambda x: f'rgba{tuple(x)}', color))
else:
  color = hue_name if hue_name is not None else None
if opacity_col is not None:
  plotly_fig.update_traces(marker=dict(color=color))

## 4. dropdown menu
- это задание явно underrated и является аналогом главного босса (но для графиков):
1. plotly: очень грустная история с тем, что любые кнопки по факту никакие не динамические и даже не имеют доступа к значению другой кнопки! (то есть это не callback-функционал, ну как минимум во всех примерах в интернете это так). поэтому пришлось удалить color=hue_name (опять же, это делает необьяснимые коллизии и мешает все цвета) и для каждой из передаваемых hue_cols установить `marker.color` = array(floats) or array(hex_colors_strings).\
важно: hue_cols переписывает hue (см. коллизии), а также автоматически идентифицирует колонки как категориальные если `nunique<=min(24, cat_trsh)`
2. bokeh: `You are generating standalone HTML/JS output, but trying to use real Python
callbacks. This combination cannot work. Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh`. другими словами, callbacks для fropdown должны быть написаны на javascript, и день попыток это осуществить так и не увенчался успехом; самый непонятный момент: как в коде javascript'а обновить глобальную переменную (для того чтобы вторая кнопка могла ее использовать)? мы с gpt так и не справились :( 

- тк реализована часть с plotly показаны только 3d графики
- чтобы появились цвета необходимо тыкнуть любой `color`
- чтобы убрать градиентный бар справа можно выбрать `color=None`

In [4]:
hue_cols = ['sex', 'length', 'diameter', 'height', 'weight', 'shucked_weight', 'viscera_weight', 'shell_weight', 'age']
mapper_dict = {
    'tsne 3D': {
        'params': {
           'n_jobs': 1,
           'verbose': False,
           'n_components': 3
        },
        'func': make_tsne,

        'features': ['weight', 'shucked_weight', 'shell_weight'],
        'hover_data': ['age', 'sex', 'weight'],
        'hue_cols': hue_cols,
        'cat_trsh': 10,
    },
    'UMAP 3D': {
      'params': {
        'n_neighbors': 40,
        'min_dist': 0.1,
        'n_jobs': 1,
        'verbose': False,
        'n_components': 3
      },
      'func': make_umap,

      'features': ['weight', 'shucked_weight', 'shell_weight'],
      'hover_data': ['age', 'sex', 'weight'],
      'hue_cols': hue_cols,
      'cat_trsh': 10,
    }, 
    'PCA 3D': {
        'params': {
           'n_components': 3
        },
        'func': make_pca,
        
        'features': ['weight', 'shucked_weight', 'shell_weight'],
        'hover_data': ['age', 'sex', 'weight'],
        'hue_cols': hue_cols,
        'cat_trsh': 10,
    },
    'SPECTRAL 3D': {
        'params': {
           'n_components': 3
        },
        'func': make_spectral,

        'features': ['weight', 'shucked_weight', 'shell_weight'],
        'hover_data': ['age', 'sex', 'weight'],
        'hue_cols': hue_cols,
        'cat_trsh': 10,
    },
}

drp.plot_dim_reduction(
  data = tr.sample(1_000),
  mapper_dict=mapper_dict,
  default_features=tr.drop(columns=['sex', 'age']).columns,
  default_hue_info=None,
  row_height=450,
  row_width=1200,
  reuse_results=False
)

100%|██████████| 4/4 [00:17<00:00,  4.50s/it]


Loading BokehJS ...

    'data': [{'customdata': array([[nan, nan, nan],
             …